Author: Ronny F. Pacheco Date: Jul 2024
Copyright: © 2024 Ronny Pacheco License: MIT License

---

MIT License

Copyright (c) 2024 Ronny Pacheco

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Needed mmodules

In [63]:
# Import neede packages
import pickle
import pandas as pd
import json

In [64]:
## https://kioku-space.com/en/jupyter-skip-execution/
from IPython.core.magic import register_cell_magic # type: ignore

@register_cell_magic
def skip(line, cell):
    return

# Pickle save

In [65]:
%%skip
# =============================================================================
# Save the variables
# =============================================================================
variables_dict = {
}

# =============================================================================
# main function
# =============================================================================
def data_save_load(option, dict_variables=None):
    """
    This function is used to save or load data for the jupyter notebook
    """
    path_folder = "ipynb_db"  # Folder to save variables
    os.makedirs(path_folder, exist_ok=True)  # Create folder if not exist
    path_file = os.path.join(path_folder, "variables.pkl") # Path to save the variables

    if option == "save":
        with open(path_file, "wb") as f:
            pickle.dump(dict_variables, f)
    elif option == "load":
        with open(path_file, "rb") as f:
            variables = pickle.load(f)
        # Now load the variables
        for key, value in variables.items():
            variables[key] = value

# =============================================================================
# Call the function
# =============================================================================
data_save_load(option="save",
               dict_variables=variables_dict)

# 1. Prepare data

In [66]:
# Load the JSON dict into a Python dictionary
path_json_file = "./data/sider_gtf_relation.json"

with open(path_json_file, "r") as f:
    dict_sider_gtf = json.load(f)

In [67]:
# Check the size of the dictionary
print(f"Size of the dictionary: {len(dict_sider_gtf)} elements")

Size of the dictionary: 2132 elements


# 2. Create table

In [74]:
def gene_finder(sublist):
    """
    This function is used to find the gene name from the sublist without repetition
    """
    gene_name = ""
    if len(sublist) == 0:
        gene_name = "intergenic"
    else:
        gene_list = []
        for elem in sublist:
            if elem.get("gene_id") is not None:
                gene_list.append(elem.get("gene_id"))
        gene_list = list(set(gene_list))  # Remove duplicates
        gene_name = ", ".join(gene_list)
    return gene_name

def transcript_finder(sublist):
    """
    This function is used to find the transcript name from the sublist without repetition
    """
    transcript_dict = {}
    transcript = False
    cds = False
    utr3 = False
    utr5 = False
    for elem in sublist:
        # Take transcript data
        if elem.get("transcript_id") is not None:
            # Create a key with the transcript_id if it does not exist already
            if transcript_dict.get(elem.get("transcript_id")) is None:
                transcript_dict[elem.get("transcript_id")] = []
            transcript = True
        # Now let's go for the CDS, 3utr and 5utr
        if elem.get("CDS") == True:
            transcript_dict[elem.get("parent_id")].append("CDS")
            cds = True
        if elem.get("3utr") == True:
            transcript_dict[elem.get("parent_id")].append("3utr")
            utr3 = True
        if elem.get("5utr") == True:
            transcript_dict[elem.get("parent_id")].append("5utr")
            utr5 = True
    
    return transcript_dict, transcript, cds, utr3, utr5

    

In [77]:
# Let's create a table where one column is the "sider name" and the other the "gene_name".
# If there si no gene_name, its intergenic region, so we will put "intergenic" in the gene_name column
list_data = []
for key, value in dict_sider_gtf.items():
    sider_name = key
    
    # Find the gen_id for each sider name
    gene_name = gene_finder(value)
    # list_data.append([sider_name, gene_name])

    # Find the transcript_id for each sider name
    transcript_name = ""
    transcript, cds, utr3, utr5 = False, False, False, False
    if gene_name != "intergenic":
        transcript_name, transcript, cds, utr3, utr5 = transcript_finder(value)
    else:
        transcript_name = "intergenic"
    list_data.append([sider_name, gene_name, transcript_name, transcript, cds, utr3, utr5])

# Create a pandas dataframe
df_sider_gtf = pd.DataFrame(list_data, columns=["sider_name", "gene_name", "transcript_name", "transcript", "cds", "utr3", "utr5"])

In [78]:
print(df_sider_gtf.shape)
print(df_sider_gtf.dtypes)
df_sider_gtf.head()

(2132, 7)
sider_name         object
gene_name          object
transcript_name    object
transcript           bool
cds                  bool
utr3                 bool
utr5                 bool
dtype: object


,sider_name,gene_name,transcript_name,transcript,cds,utr3,utr5
0,src_c01.10,intergenic,intergenic,False,False,False,False
1,src_c01.20A,LINF_010005800,{'LINF_01T0005800': ['3utr']},True,False,True,False
2,src_c01.30A,LINF_010006300,{'LINF_01T0006300': ['3utr']},True,False,True,False
3,src_c01.40,LINF_010006600,{'LINF_01T0006600': ['3utr']},True,False,True,False
4,src_c01.50A,LINF_010007050,{'LINF_01T0007050': []},True,False,False,False
